## vgg块

In [6]:
import torch.nn as nn
import torch

def vgg_block(in_channels, out_channels, num_conv_layers):
    layers = []
    for _ in range(num_conv_layers):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU(inplace=True))
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

class VGG11(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv_layers = nn.Sequential(
            vgg_block(in_channels, 64, 1),      # conv1
            vgg_block(64,512,2)
#             vgg_block(64, 128, 1),            # conv2
#             vgg_block(128, 256, 2),           # conv3
#             vgg_block(256, 512, 2),           # conv4
#             vgg_block(512, 512, 2)            # conv5
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )
        self.softmax = nn.Softmax()
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        x = self.softmax(x)
        return x


In [7]:
import torch
import pandas as pd
import torch.utils.data as Data
import torch.nn.functional as F

batch_sizes = 4

df = pd.read_csv('fashion-mnist_train.csv')

x_train = df.iloc[:100,1:]
y_train = df.iloc[:100,:1]

x_train_tensor = torch.FloatTensor(x_train.values)
y_train_tensor = torch.FloatTensor(y_train.values)

train_dataset = Data.TensorDataset(x_train_tensor, y_train_tensor)

train_dataloader = Data.DataLoader(train_dataset,batch_sizes,shuffle=True)
len(train_dataloader)


25

In [10]:
import torch.nn as nn
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss = nn.MSELoss()
model = VGG11(1,10).to(device)
optimizer = optim.Adam(model.parameters(),lr = 1e-3)


In [11]:
# 训练模型
def train_model(model,dataloader):
    model.train()
    
    total_loss = 0
    for idx,(x,y) in enumerate(dataloader):
        x = x.view(batch_sizes,1,28,28)
        y_pred = model(x)
        cur_loss = loss(y_pred,y)
        optimizer.zero_grad()
        cur_loss.backward()
        optimizer.step()
        
        total_loss += cur_loss.item()
    print(f"train loss:{total_loss/len(train_dataset)}")
    

In [12]:
Epoch = 10
for i in range(Epoch):
    train_model(model,train_dataloader)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [ ]:
x = torch.randn(2,1)
m = nn.Linear(1,3)
y = m(x)
print(x)
print(y)